In [1]:
from monai.transforms import Activations, ScaleIntensity, EnsureType, LoadImage
from monai.transforms import Compose, AddChannel, Resize, RandRotate90d
from monai.transforms import LoadImaged, AddChanneld, ScaleIntensityd, RandAffined, EnsureTyped, RandShiftIntensityd
from monai.transforms import RandRotated, Resized, SqueezeDimd, CropForegroundd, RandCropByPosNegLabeld
from monai.transforms import MaskIntensityd, AsDiscrete

import argparse
import sys
import monai
import torch
from monai.utils import set_determinism

from utils import getDevice
from reg_model import getRegistrationModel
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from transforms_dict import getRegistrationEvalTransformsForMRI, getRegistrationEvalTransformsForAtlas
import time
import torchinfo


from torch.nn import Softmax

from transforms import CropMRId, BinaryMask, GetLargestComponent, GetLabelsAsOneHotd, Shaped, GetMaxChannelWise
from transforms import N4MRI, LoadNibabel, NibabelToNumpy, ResampleMRIToAtlas, ToNibabel, SaveNibabel, InverseOneHot
from transforms import MaskIntensityMultid
from utils import getAffineHeader, try_load, getDevice

from reg_model import DeformableNet

In [2]:
set_determinism(seed=0)

device = getDevice()

registration_type = 'affine'
modelname = "registration/atlas_deformable_feminad_1-1-1.pth"
mriname = "Feminad_test.nii.gz"

model = DeformableNet(128)

checkpoint = torch.load('./models/' + str(modelname), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
model.eval()
#print(model)


atlas_name = "dataset2/Atlas/Atlas_MRI_128.nii.gz"
mri_transform = Compose(
        [
            LoadNibabel(),
            ResampleMRIToAtlas(atlas=atlas_name, mask=False, resample=True),
            NibabelToNumpy(),
            AddChannel(),
            ScaleIntensity(minv=0.0, maxv=1.0),
            AddChannel(),
            EnsureType(),
        ]
    )
    
atlas_transform = Compose(
        [
            LoadImage(image_only=True, as_closest_canonical=True),
            AddChannel(),
            AddChannel(),
            ScaleIntensity(minv=0.0, maxv=1.0),
            EnsureType(),
        ]
    )

mri = mri_transform(mriname)
atlas = atlas_transform(atlas_name)

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


model.globalnet.output_block.fc.register_forward_hook(get_activation('fc'))



start = time.process_time()
with torch.no_grad():
    data = torch.cat((mri, atlas), dim=1)       
    data = {"fixed_image": atlas, "moving_image": mri}
    
    ddf = model(data)
    #print(model)
    print(activation['fc'])
    
    #mri_save_transform(pred_image)
    #mri_inverse_transform(pred_image)
print(time.process_time() - start)

monai.networks.blocks.Warp: Using PyTorch native grid_sample.


tensor([[ 9.6100e-01,  1.3745e-03, -2.6519e-02, -7.3440e+00,  2.1075e-02,
          1.0276e+00,  9.2076e-02, -4.7933e+00, -4.5888e-02, -3.7800e-02,
          9.7728e-01,  1.6427e+01]], device='cuda:0')
Metadata
	affine: tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], dtype=torch.float64)

Applied operations
[]
Is batch?: False
1.2357184300000004
